In [ ]:
import numpy as np    # Numeric and matrix computation
import pandas as pd   # Optional: good package for manipulating data 
import sklearn as sk  # Package with learning algorithms implemented
from sklearn.model_selection import train_test_split, GridSearchCV # split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

## Hecho por Hasnain Shafqat

**Read CSV**

In [ ]:
df = pd.read_csv("preprocessed_mean.csv");
df = pd.get_dummies(df, columns=["mode","key"])

targetCol = 'music_genre';
Y = df[targetCol]
X = df.drop([targetCol],axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state = 33)
df.shape

(20000, 26)

**Default Random Forest Classifier**

In [ ]:
RFdefault = RandomForestClassifier()
RFdefault.fit(X_train,Y_train)
scoredefalt = RFdefault.score(X_test, Y_test)
print(f"accuracy {scoredefalt}")

accuracy 0.901


**Finding Best hyperparameters for random forest**

In [ ]:
cv= 10
crit = ['gini', 'entropy']
max_d = [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]
max_features =['auto', 'sqrt', 'log2']
params = {'n_estimators': [1,2,5,10,20,50,100,200, 250, 300, 350, 400], 'criterion':crit, 'max_depth' : max_d, 'max_features' : max_features}

Randomforest = RandomForestClassifier()

gridRandom= GridSearchCV(Randomforest, params, cv=cv, n_jobs=-1)
gridRandom.fit(X_train,Y_train)
score = gridRandom.score(X_test, Y_test)
print(score)
print(gridRandom.best_params_)

0.9015
{'criterion': 'entropy', 'max_depth': 90, 'max_features': 'sqrt', 'n_estimators': 250}


**Checking if restricting is a good idea**

In [ ]:
RF1 = RandomForestClassifier(criterion = 'entropy', max_depth =  90, max_features =  'sqrt', n_estimators = 250)
RF1.fit(X_train,Y_train)
score = RF1.score(X_test, Y_test)
print(f"score limitando {score}")
RF2 = RandomForestClassifier(criterion = 'entropy', max_features =  'sqrt', n_estimators = 250)
RF2.fit(X_train,Y_train)
score2 = RF2.score(X_test, Y_test)
print(f"score sin limitar {score2}")

score limitando 0.9035
score sin limitar 0.902


**Final Model**

In [ ]:
knRandom = RandomForestClassifier(criterion = 'entropy', max_depth =  90, max_features =  'sqrt', n_estimators = 250)
knRandom.fit(X_train, Y_train)
pred=knRandom.predict(X_test)
print("\nConfusion matrix on test set:\n",sk.metrics.confusion_matrix(Y_test, pred))


Confusion matrix on test set:
 [[893  55   1   6]
 [ 18 939  39  61]
 [  0   9 897  89]
 [  8  19  83 883]]


In [ ]:
Y_test_pred = knRandom.predict(X_test);
print(sk.metrics.classification_report(Y_test,Y_test_pred))

              precision    recall  f1-score   support

   Classical       0.97      0.94      0.95       955
  Electronic       0.92      0.89      0.90      1057
     Hip-Hop       0.88      0.90      0.89       995
        Rock       0.85      0.89      0.87       993

    accuracy                           0.90      4000
   macro avg       0.90      0.90      0.90      4000
weighted avg       0.90      0.90      0.90      4000

